In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import neuralnetwork as nn
from tfomics import neuralbuild as nb
from tfomics import utils, learn, explore


In [2]:
filename = 'processed_dataset.hdf5'
#data_path = 'D:/Dropbox/TFconnect'
data_path = '/home/peter/Code/tensorflow/data'
filepath = os.path.join(data_path,filename)

group_name = ['processed_data']
dataset = h5py.File(filepath,'r')
%time dtf = np.array(dataset['/'+group_name[0]+'/dtf'])
ltf = np.array(dataset['/'+group_name[0]+'/ltf'])
dtf_crossval = np.array(dataset['/'+group_name[0]+'/dtf_crossval'])
ltf_crossval = np.array(dataset['/'+group_name[0]+'/ltf_crossval'])

X_train = dtf.transpose([0,2,3,1]) #[:10000,:,:,:]
y_train = ltf#[:10000,:]
X_valid = dtf_crossval.transpose([0,2,3,1]) #[:5000,:,:,:]
y_valid = ltf_crossval#[:5000,:]

CPU times: user 440 ms, sys: 1.06 s, total: 1.5 s
Wall time: 1.5 s


In [3]:

def model(input_shape, num_labels):


    # create model
    layer1 = {  'layer': 'input',
                'input_shape': input_shape,
              }
    layer2 = {  'layer': 'conv1d', 
                'num_filters': {'start': 32, 'bounds': [1, 200], 'scale': 25, 'multiples': 4},
                'filter_size': {'start': 7, 'bounds': [5, 32], 'scale': 5, 'odd': True, 'multiples': 2},
                'norm': 'batch',
                'padding': 'SAME',
                'activation': 'relu',
                'dropout': {'start': 0.2, 'bounds': [0., 0.6], 'scale': 0.1},
                'name': 'conv1'
             }
    layer3 = {  'layer': 'conv1d_residual', 
                'filter_size': {'start': 5, 'bounds': [3, 9], 'scale': 4, 'odd': True},
                'activation': 'relu',
                'pool_size': {'start': 10, 'bounds': [1, 200], 'scale': 30, 'multiples': 2},
                'dropout': {'start': .2, 'bounds': [0., .6], 'scale': .1},
                'name': 'conv1_resid'
              }
    layer4 = {  'layer': 'conv1d', 
                'num_filters': {'start': 64, 'bounds': [1, 200], 'scale': 25, 'multiples': 4},
                'filter_size': {'start': 7, 'bounds': [5, 32], 'scale': 5, 'odd': True, 'multiples': 2},
                'norm': 'batch',
                'padding': 'SAME',
                'activation': 'relu',
                'dropout': {'start': 0.2, 'bounds': [0., 0.6], 'scale': 0.1},
                'name': 'conv2'
             }
    layer5 = {  'layer': 'conv1d_residual', 
                'filter_size': {'start': 5, 'bounds': [3, 9], 'scale': 4, 'odd': True},
                'activation': 'relu',
                'pool_size': {'start': 10, 'bounds': [1, 200], 'scale': 30, 'multiples': 2},
                'dropout': {'start': .2, 'bounds': [0., .6], 'scale': .1},
                'name': 'conv2_resid'
              }              
    layer6 = {  'layer': 'dense', 
                'num_units': {'start': 256, 'bounds': [16, 1000], 'scale': 50, 'multiples': 4},
                #'norm': 'batch',
                'activation': 'relu',
                'dropout': {'start': 0.4, 'bounds': [0., 0.8], 'scale': 0.2},
                'name': 'dense1'
              }            
    layer7 = {  'layer': 'dense', 
                'num_units': num_labels,
                'activation': 'softmax',
              }

    #from tfomics import build_network
    model_layers = [layer1, layer2, layer3, layer4, layer5, layer6, layer7]

    # optimization parameters
    optimization = {"objective": "categorical",
                    "optimizer": "adam",
                    "learning_rate": 0.001,
                    "l2": 1e-6
                    #"learning_rate": {'start': -3, 'bounds': [-4, -1], 'scale': 1.5, 'transform': 'log'},      
                    #"l2": {'start': -6, 'bounds': [-8, -2], 'scale': 3, 'transform': 'log'},
                    # "l1": 0, 
                    }
    return model_layers, optimization

# build network
input_shape = list(X_train.shape)
input_shape[0] = None
num_labels = y_train.shape[1]
model_layers, optimization = model(input_shape, num_labels)

# build neural opti
optimizer = explore.NeuralOptimizer(model_layers, optimization)


In [4]:
optimizer.sample_network()

[{'input_shape': [None, 330, 1, 3], 'layer': 'input'},
 {'activation': 'relu',
  'dropout': 0.23291182036441585,
  'filter_size': 9,
  'layer': 'conv1d',
  'name': 'conv1',
  'norm': 'batch',
  'num_filters': 72,
  'padding': 'SAME'},
 {'activation': 'relu',
  'dropout': 0.3334532694303734,
  'filter_size': 5,
  'layer': 'conv1d_residual',
  'name': 'conv1_resid',
  'pool_size': 34},
 {'activation': 'relu',
  'dropout': 0.3482890601427887,
  'filter_size': 5,
  'layer': 'conv1d',
  'name': 'conv2',
  'norm': 'batch',
  'num_filters': 24,
  'padding': 'SAME'},
 {'activation': 'relu',
  'dropout': 0.16019423256229898,
  'filter_size': 7,
  'layer': 'conv1d_residual',
  'name': 'conv2_resid',
  'pool_size': 12},
 {'activation': 'relu',
  'dropout': 0.2921285704615663,
  'layer': 'dense',
  'name': 'dense1',
  'num_units': 232},
 {'activation': 'softmax', 'layer': 'dense', 'num_units': 2}]

In [5]:
# optimize model parameters
train = {'inputs': [X_train], 'targets': y_train}
valid = {'inputs': [X_valid], 'targets': y_valid}        
optimizer.optimize(train, valid, num_trials=100, num_epochs=20, batch_size=128, verbose=0)


---------------------------------------------------------
Running baseline model

Model layers:
name: conv1
filter_size: 7
num_filters: 32
dropout: 0.2
name: conv1_resid
pool_size: 10
filter_size: 5
dropout: 0.2
name: conv2
filter_size: 7
num_filters: 64
dropout: 0.2
name: conv2_resid
pool_size: 10
filter_size: 5
dropout: 0.2
dropout: 0.4
name: dense1
num_units: 256
num_units: 2

Optimization:
learning_rate: 0.001
l2: 1e-06

Epoch 1 out of 20 


KeyboardInterrupt: 

In [ ]:
# save optimal model
optimizer.print_optimal_model()